# Two-phase flow model with finite difference discretization

Reading packages

In [1]:
from sympy import Symbol, Function, sin, Eq, simplify, expand, factor, collect, solve, differentiate_finite

Defining Coordinates

In [43]:
t = Symbol('t')
ti = Symbol('t_i')
tip1 = Symbol('t_i+1')
tim1 = Symbol('t_i-1')

x = Symbol('x')
x0 = Symbol('x_0')
x1 = Symbol('x_1')
xNm1 = Symbol('x_N-1')
xN = Symbol('x_N')
xj = Symbol('x_j')
xjm1 = Symbol('x_j-1')
xjp1 = Symbol('x_j+1')

Defining state Variables

In [36]:
alphaL = Function('alpha_L')
alphaG = Function('alpha_G')
P = Function('P')
qL = Function('q_L')
qG = Function('q_G')

Defining Parameters

In [4]:
A = Symbol('A')
g = Symbol('g')
theta = Symbol('theta')

Defining auxiliar symbols

In [5]:
rhoL = Function('rho_L')
rhoG = Function('rho_G')
vL = Function('v_L')
vG = Function('v_G')
dPL = Function('\Delta P_L')
dPG = Function('\Delta P_G')
sourceL = Function('S_L')
sourceG = Function('S_G')

Auxiliar functions

## Dynamic Model

### Mass balance for liquid phase

In [34]:
print("Mass Balance L")
BMA_L = Eq(
    differentiate_finite(rhoL(t,xj)*alphaL(t,xj), t, points=[ti, tip1]),
    - 1/(2*A) * (
        differentiate_finite(qL(ti,x), x, points=[xjm1, xjp1]) +
        differentiate_finite(qL(tip1,x), x, points=[xjm1, xjp1])
    )
)
BMA_L = simplify(BMA_L)
BMA_L.doit()

Mass Balance L


Eq((alpha_L(t_i, x_j)*rho_L(t_i, x_j) - alpha_L(t_i+1, x_j)*rho_L(t_i+1, x_j))/(t_i - t_i+1), (-q_L(t_i, x_j+1) + q_L(t_i, x_j-1) - q_L(t_i+1, x_j+1) + q_L(t_i+1, x_j-1))/(2*A*(x_j+1 - x_j-1)))

### Mass balance for gas phase

In [37]:
print("Mass Balance G")
BMA_G = Eq(
    differentiate_finite(rhoG(t,xj)*alphaG(t,xj), t, points=[ti, tip1]),
    - 1/(2*A) * (
        differentiate_finite(qG(ti,x), x, points=[xjm1, xjp1]) +
        differentiate_finite(qG(tip1,x), x, points=[xjm1, xjp1])
    )
)
BMA_G = simplify(BMA_G)
BMA_G.doit()

Mass Balance G


Eq((alpha_G(t_i, x_j)*rho_G(t_i, x_j) - alpha_G(t_i+1, x_j)*rho_G(t_i+1, x_j))/(t_i - t_i+1), (-q_G(t_i, x_j+1) + q_G(t_i, x_j-1) - q_G(t_i+1, x_j+1) + q_G(t_i+1, x_j-1))/(2*A*(x_j+1 - x_j-1)))

### Momentum Balance for Liquid Phase

In [41]:
BMM_L = Eq(
    differentiate_finite(qL(t,xj), t, points=[ti, tip1]), 
    - A / 2 * alphaL(ti,xj) * (
        differentiate_finite(P(ti,x), x, points=[xjm1, xjp1]) +
        differentiate_finite(P(tip1,x), x, points=[xjm1, xjp1])
    )
    - A / 2 * dPL(ti,xj) * (
        differentiate_finite(alphaL(ti,x), x, points=[xjm1, xjp1]) +
        differentiate_finite(alphaL(tip1,x), x, points=[xjm1, xjp1])
    )
    - A /2 * (
        differentiate_finite(alphaL(ti,x)*(vL(ti,x))**2*rhoL(ti,x), x, points=[xjm1, xjp1]) +
        differentiate_finite(alphaL(tip1,x)*(vL(tip1,x))**2*rhoL(tip1,x), x, points=[xjm1, xjp1])
    )
    + A * sourceL(ti,xj)
)
BMM_L = simplify(BMM_L)
BMM_L.doit()

Eq((q_L(t_i, x_j) - q_L(t_i+1, x_j))/(t_i - t_i+1), A*(2*(x_j+1 - x_j-1)*S_L(t_i, x_j) - (P(t_i, x_j+1) - P(t_i, x_j-1) + P(t_i+1, x_j+1) - P(t_i+1, x_j-1))*alpha_L(t_i, x_j) - (alpha_L(t_i, x_j+1) - alpha_L(t_i, x_j-1) + alpha_L(t_i+1, x_j+1) - alpha_L(t_i+1, x_j-1))*\Delta P_L(t_i, x_j) - alpha_L(t_i, x_j+1)*rho_L(t_i, x_j+1)*v_L(t_i, x_j+1)**2 + alpha_L(t_i, x_j-1)*rho_L(t_i, x_j-1)*v_L(t_i, x_j-1)**2 - alpha_L(t_i+1, x_j+1)*rho_L(t_i+1, x_j+1)*v_L(t_i+1, x_j+1)**2 + alpha_L(t_i+1, x_j-1)*rho_L(t_i+1, x_j-1)*v_L(t_i+1, x_j-1)**2)/(2*(x_j+1 - x_j-1)))

### Momentum Balance for Gas Phase

In [42]:
BMM_G = Eq(
    differentiate_finite(qG(t,xj), t, points=[ti, tip1]), 
    - A / 2 * alphaG(ti,xj) * (
        differentiate_finite(P(ti,x), x, points=[xjm1, xjp1]) +
        differentiate_finite(P(tip1,x), x, points=[xjm1, xjp1])
    )
    - A / 2 * dPG(ti,xj) * (
        differentiate_finite(alphaG(ti,x), x, points=[xjm1, xjp1]) +
        differentiate_finite(alphaG(tip1,x), x, points=[xjm1, xjp1])
    )
    - A /2 * (
        differentiate_finite(alphaG(ti,x)*(vG(ti,x))**2*rhoG(ti,x), x, points=[xjm1, xjp1]) +
        differentiate_finite(alphaG(tip1,x)*(vG(tip1,x))**2*rhoG(tip1,x), x, points=[xjm1, xjp1])
    )
    + A * sourceG(ti,xj)
)
BMM_G = simplify(BMM_G)
BMM_G.doit()

Eq((q_G(t_i, x_j) - q_G(t_i+1, x_j))/(t_i - t_i+1), A*(2*(x_j+1 - x_j-1)*S_G(t_i, x_j) - (P(t_i, x_j+1) - P(t_i, x_j-1) + P(t_i+1, x_j+1) - P(t_i+1, x_j-1))*alpha_G(t_i, x_j) - (alpha_G(t_i, x_j+1) - alpha_G(t_i, x_j-1) + alpha_G(t_i+1, x_j+1) - alpha_G(t_i+1, x_j-1))*\Delta P_G(t_i, x_j) - alpha_G(t_i, x_j+1)*rho_G(t_i, x_j+1)*v_G(t_i, x_j+1)**2 + alpha_G(t_i, x_j-1)*rho_G(t_i, x_j-1)*v_G(t_i, x_j-1)**2 - alpha_G(t_i+1, x_j+1)*rho_G(t_i+1, x_j+1)*v_G(t_i+1, x_j+1)**2 + alpha_G(t_i+1, x_j-1)*rho_G(t_i+1, x_j-1)*v_G(t_i+1, x_j-1)**2)/(2*(x_j+1 - x_j-1)))

## Steady-State Model

### Mass balance for liquid phase

In [22]:
print("Mass Balance L")
BMA_L = Eq( qL(0,xjm1) , qL(0,xjp1) )
BMA_L.doit()

Mass Balance L


Eq(q_L(0, x_j-1), q_L(0, x_j+1))

In [48]:
print("For j=0")
BMA_L0 = Eq( qL(0,x0) , qL(0,x1) )
BMA_L0.doit()

For j=0


Eq(q_L(0, x_0), q_L(0, x_1))

In [47]:
print("For j=N")
BMA_LN = Eq( qL(0,xNm1) , qL(0,xN) )
BMA_LN.doit()

For j=N


Eq(q_L(0, x_N-1), q_L(0, x_N))

### Mass balance for gas phase

In [45]:
print("For j=0")
BMA_G0 = Eq( qG(0,x0) , qG(0,x1) )
BMA_G0.doit()

For N=0


Eq(q_G(0, x_0), q_G(0, x_1))

In [50]:
print("For 0<j<N")
BMA_G = Eq( qG(0,xjm1) , qG(0,xjp1) )
BMA_G.doit()

For 0<j<N


Eq(q_G(0, x_j-1), q_G(0, x_j+1))

In [51]:
print("For j=N")
BMA_GN = Eq( qG(0,xNm1) , qG(0,xN) )
BMA_GN.doit()

For j=N


Eq(q_G(0, x_N-1), q_G(0, x_N))

### Momentum Balance for Liquid Phase

In [52]:
print("For j=0")
BMM_L0 = Eq(
    sourceL(0,x0),
    alphaL(0,x0) * (
        differentiate_finite(P(0,x), x, points=[x0, x1])
    ) +
    dPL(0,x0)*(
        differentiate_finite(alphaL(0,x), x, points=[x0, x1])
    ) + 
    (
        differentiate_finite(alphaL(0,x)*(vL(0,x))**2*rhoL(0,x), x, points=[x0, x1])
    ) 
)
BMM_L0 = simplify(BMM_L0)
BMM_L0.doit()

For j=0


Eq(S_L(0, x_0), ((P(0, x_0) - P(0, x_1))*alpha_L(0, x_0) + (alpha_L(0, x_0) - alpha_L(0, x_1))*\Delta P_L(0, x_0) + alpha_L(0, x_0)*rho_L(0, x_0)*v_L(0, x_0)**2 - alpha_L(0, x_1)*rho_L(0, x_1)*v_L(0, x_1)**2)/(x_0 - x_1))

In [53]:
print("For 0<j<N")
BMM_L = Eq(
    sourceL(0,xj),
    alphaL(0,xj) * (
        differentiate_finite(P(0,x), x, points=[xjm1, xjp1])
    ) +
    dPL(0,xj)*(
        differentiate_finite(alphaL(0,x), x, points=[xjm1, xjp1])
    ) + 
    (
        differentiate_finite(alphaL(0,x)*(vL(0,x))**2*rhoL(0,x), x, points=[xjm1, xjp1])
    ) 
)
BMM_L = simplify(BMM_L)
BMM_L.doit()

For 0<j<N


Eq(2*S_L(0, x_j), ((P(0, x_j+1) - P(0, x_j-1))*alpha_L(0, x_j) + (alpha_L(0, x_j+1) - alpha_L(0, x_j-1))*\Delta P_L(0, x_j) + alpha_L(0, x_j+1)*rho_L(0, x_j+1)*v_L(0, x_j+1)**2 - alpha_L(0, x_j-1)*rho_L(0, x_j-1)*v_L(0, x_j-1)**2)/(x_j+1 - x_j-1))

In [54]:
print("For j=N")
BMM_LN = Eq(
    sourceL(0,xN),
    alphaL(0,xN) * (
        differentiate_finite(P(0,x), x, points=[xNm1, xN])
    ) +
    dPL(0,xN)*(
        differentiate_finite(alphaL(0,x), x, points=[xNm1, xN])
    ) + 
    (
        differentiate_finite(alphaL(0,x)*(vL(0,x))**2*rhoL(0,x), x, points=[xNm1, xN])
    ) 
)
BMM_LN = simplify(BMM_LN)
BMM_LN.doit()

For j=N


Eq(S_L(0, x_N), ((P(0, x_N) - P(0, x_N-1))*alpha_L(0, x_N) + (alpha_L(0, x_N) - alpha_L(0, x_N-1))*\Delta P_L(0, x_N) + alpha_L(0, x_N)*rho_L(0, x_N)*v_L(0, x_N)**2 - alpha_L(0, x_N-1)*rho_L(0, x_N-1)*v_L(0, x_N-1)**2)/(x_N - x_N-1))

### Momentum Balance for Gas Phase

In [55]:
print("For j=0")
BMM_G0 = Eq(
    sourceG(0,x0),
    alphaG(0,x0) * (
        differentiate_finite(P(0,x), x, points=[x0, x1])
    ) +
    dPG(0,x0)*(
        differentiate_finite(alphaG(0,x), x, points=[x0, x1])
    ) + 
    (
        differentiate_finite(alphaG(0,x)*(vG(0,x))**2*rhoG(0,x), x, points=[x0, x1])
    ) 
)
BMM_G0 = simplify(BMM_G0)
BMM_G0.doit()

For j=0


Eq(S_G(0, x_0), ((P(0, x_0) - P(0, x_1))*alpha_G(0, x_0) + (alpha_G(0, x_0) - alpha_G(0, x_1))*\Delta P_G(0, x_0) + alpha_G(0, x_0)*rho_G(0, x_0)*v_G(0, x_0)**2 - alpha_G(0, x_1)*rho_G(0, x_1)*v_G(0, x_1)**2)/(x_0 - x_1))

In [25]:
BMM_G = Eq(
    sourceG(0,xj),
    (1-alphaL(0,xj)) * (
        differentiate_finite(P(0,x), x, points=[xjm1, xjp1])
    ) +
    dPG(0,xj)*(
        differentiate_finite((1-alphaL(0,x)), x, points=[xjm1, xjp1])
    ) +
    (
        differentiate_finite((1-alphaL(0,x))*(vG(0,x))**2*rhoG(0,x), x, points=[xjm1, xjp1])
    ) 
)
BMM_G = simplify(BMM_G)
BMM_G.doit()

Eq(S_G(0, x_j), (-(P(0, x_j+1) - P(0, x_j-1))*(alpha_L(0, x_j) - 1) - (alpha_L(0, x_j+1) - 1)*rho_G(0, x_j+1)*v_G(0, x_j+1)**2 - (alpha_L(0, x_j+1) - alpha_L(0, x_j-1))*\Delta P_G(0, x_j) + (alpha_L(0, x_j-1) - 1)*rho_G(0, x_j-1)*v_G(0, x_j-1)**2)/(x_j+1 - x_j-1))

In [56]:
print("For j=N")
BMM_GN = Eq(
    sourceG(0,xN),
    alphaG(0,xN) * (
        differentiate_finite(P(0,x), x, points=[xNm1, xN])
    ) +
    dPG(0,xN)*(
        differentiate_finite(alphaG(0,x), x, points=[xNm1, xN])
    ) + 
    (
        differentiate_finite(alphaG(0,x)*(vG(0,x))**2*rhoG(0,x), x, points=[xNm1, xN])
    ) 
)
BMM_GN = simplify(BMM_GN)
BMM_GN.doit()

For j=N


Eq(S_G(0, x_N), ((P(0, x_N) - P(0, x_N-1))*alpha_G(0, x_N) + (alpha_G(0, x_N) - alpha_G(0, x_N-1))*\Delta P_G(0, x_N) + alpha_G(0, x_N)*rho_G(0, x_N)*v_G(0, x_N)**2 - alpha_G(0, x_N-1)*rho_G(0, x_N-1)*v_G(0, x_N-1)**2)/(x_N - x_N-1))